In [1]:
import scipy.io as sio
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
Vv_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\downloaded_pp_features\\gpds_signet\\validation'
Lc_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\downloaded_pp_features\\gpds_signet\\development'

In [3]:
data_f = pd.read_csv(os.path.join(Vv_folder,'data_features.csv'))
visual_f = pd.read_csv(os.path.join(Vv_folder,'visual_features.csv'))
dev_data_f = pd.read_csv(os.path.join(Lc_folder,'data_features.csv'))
dev_visual_f = pd.read_csv(os.path.join(Lc_folder,'visual_features.csv'))

In [4]:
data_f.head()

,user_id,sig_id,fakeness
0,352,1,True
1,352,2,True
2,352,3,True
3,352,4,True
4,352,5,True


In [5]:
visual_f.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.000000,0.000000,0.0,1.370572,0.000000,0.000000,0.0,0.149738,0.323547,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.186365,0.331796,0.356497,0.0
1,0.160537,0.000000,0.0,1.169396,0.239608,0.000000,0.0,0.000000,1.208937,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.141376,0.893188,0.000000,0.0
2,0.393000,0.223869,0.0,0.195907,0.000000,0.000000,0.0,0.060111,0.088993,0.194043,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.038160,0.215892,0.050661,0.0
3,0.280439,0.609959,0.0,0.392170,0.000000,0.568858,0.0,0.000000,0.246439,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.138333,1.180603,0.000000,0.000000,0.0
4,0.351282,0.000000,0.0,0.000000,0.000000,0.940088,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.398407,1.215286,0.000000,0.000000,0.0


In [6]:
dev_data_f.head()

,user_id,sig_id,fakeness
0,403,1,True
1,403,2,True
2,403,3,True
3,403,4,True
4,403,5,True


In [7]:
dev_visual_f.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.000000,1.331955,0.000000,0.562717,0.000000,0.565360,0.283220,0.000000,0.0,0.000000,...,0.000000,0.0,0.374695,0.000000,0.0,0.304021,0.0,0.0,0.000000,0.0
1,0.439652,0.577330,0.000000,0.000000,0.030665,0.000000,0.657206,0.000000,0.0,0.395038,...,1.089617,0.0,1.774396,0.456928,0.0,0.262468,0.0,0.0,0.000000,0.0
2,0.000000,0.638873,0.000000,0.000000,0.000000,0.572654,0.726881,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.284252,0.0,0.0,0.358418,0.0
3,0.713571,0.489058,0.107338,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.391948,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
4,0.000000,0.821043,0.220040,0.000000,0.000000,0.022878,0.000000,0.209341,0.0,0.000000,...,0.263657,0.0,0.575950,0.000000,0.0,0.307809,0.0,0.0,0.000000,0.0


In [8]:
data_f_gen = data_f.loc[data_f['fakeness']==False]
data_f_fake = data_f.loc[data_f['fakeness']==True]
data_f_gen_12 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(1,13))]
data_f_gen_valid_12 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(13,25))] 
dev_data_f_gen = dev_data_f.loc[dev_data_f['fakeness']==False]

In [9]:
dev_data_f_gen.head()

,user_id,sig_id,fakeness
15909,403,1,False
15910,403,2,False
15911,403,3,False
15912,403,4,False
15913,403,5,False


In [ ]:
DECISION_THRESHOLD = 0.5
scores_sum = 0
run_count = 0
gens_sum = 0
gens_r_count = 0
fakes_sum = 0
fakes_r_count = 0
randoms_sum = 0
randoms_r_count = 0
for user_id in tqdm.tqdm(data_f['user_id'].unique(), ascii=True):
    clf = MLPClassifier(hidden_layer_sizes=(150,100,50),random_state=1923)
    
    y_train = pd.concat([data_f_gen_12.loc[data_f_gen_12['user_id']==user_id].reset_index(drop=True), dev_data_f.loc[dev_data_f['fakeness']==False].reset_index(drop=True)], ignore_index=True)['user_id']==user_id
    X_train = pd.concat([visual_f.loc[data_f_gen_12.loc[data_f_gen_12['user_id']==user_id].index].reset_index(drop=True),dev_visual_f.loc[dev_data_f.loc[dev_data_f['fakeness']==False].index].reset_index(drop=True)])   
    clf.fit(X_train, y_train)
    y_valid_fakes = data_f_fake.loc[(data_f_fake['user_id']==user_id)]
    y_valid_f = len(y_valid_fakes.index)*[False]
    X_valid_f = visual_f.loc[y_valid_fakes.index]
    # score = accuracy_score(y_valid_f, (clf.predict(X_valid_f)))
    score = accuracy_score(y_valid_f, (clf.predict_proba(X_valid_f)[:,1]> DECISION_THRESHOLD))
    # score = clf.score(X_valid_f,y_valid_f)
    fakes_sum += score
    fakes_r_count += 1 
    # print(score)
    scores_sum += score
    run_count += 1
    y_valid_gens = data_f_gen_valid_12.loc[data_f_gen_valid_12['user_id']==user_id]
    y_valid_g = len(y_valid_gens.index)*[True]
    X_valid_g = visual_f.loc[y_valid_gens.index]
    # score = accuracy_score(y_valid_g, (clf.predict(X_valid_g)))
    score = accuracy_score(y_valid_g, (clf.predict_proba(X_valid_g)[:,1]> DECISION_THRESHOLD))
    # score = clf.score(X_valid_g,y_valid_g)
    gens_sum += score
    gens_r_count += 1 
    # print(score)
    scores_sum += score
    run_count += 1
    y_valid_rand_fakes = data_f_gen_valid_12.loc[choice(data_f_gen_valid_12.loc[data_f_gen_valid_12['user_id']!=user_id].index,10)]
    y_valid_rf = len(y_valid_rand_fakes.index)*[False]
    X_valid_rf = visual_f.loc[y_valid_rand_fakes.index]
    score = clf.score(X_valid_rf,y_valid_rf)
    randoms_sum += score
    randoms_r_count += 1 
    # print(score)
    scores_sum += score
    run_count += 1
    # print(scores_sum/run_count)
    print ("gens_avg:", 100*(1-(float(gens_sum)/gens_r_count)), "fakes_avg:", 100*(1-(float(fakes_sum)/fakes_r_count)), 
            "cumulative_avg:", 100*(1-((float(gens_sum)/gens_r_count)/2 + (float(fakes_sum)/fakes_r_count)/2)))



  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

gens_avg: 0.0 fakes_avg: 13.33333333333333 cumulative_avg: 6.666666666666665




  2%|#6                                                                                 | 1/50 [00:12<10:12, 12.50s/it]

gens_avg: 0.0 fakes_avg: 16.666666666666664 cumulative_avg: 8.333333333333325




  4%|###3                                                                               | 2/50 [00:25<10:09, 12.69s/it]

gens_avg: 5.5555555555555465 fakes_avg: 11.111111111111105 cumulative_avg: 8.333333333333325




  6%|####9                                                                              | 3/50 [00:38<10:04, 12.85s/it]

In [ ]:
len(visual_f.loc[y_train.index])

In [ ]:
data_f_gen_12.loc[data_f_gen_12['user_id']==user_id].reset_index(drop=True)

In [ ]:
pd.concat([visual_f.loc[data_f_gen_12.loc[data_f_gen_12['user_id']==user_id].index].reset_index(drop=True),dev_visual_f.loc[dev_data_f.loc[dev_data_f['fakeness']==False].index].reset_index(drop=True)])

In [ ]:
pd.concat([data_f_gen_12.loc[data_f_gen_12['user_id']==user_id].reset_index(drop=True), dev_data_f.loc[dev_data_f['fakeness']==False].reset_index(drop=True)], ignore_index=True)['user_id']==user_id

In [ ]:
visual_f.loc[data_f_gen_12.loc[data_f_gen_12['user_id']==user_id].index]

In [ ]:
visual_f.loc[1503]